# Harmonizome ETL: Jensen Lab

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Sources:
 * http://tissues.jensenlab.org/Downloads
 * http://diseases.jensenlab.org/Downloads
 * http://compartments.jensenlab.org/Downloads

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter hide_code

{% do SectionField(
    name='data',
    title='Upload Data',
    img='load_icon.png'
) %}

{% do SectionField(
    name='settings',
    title='Settings',
    img='setting_icon.png'
) %}

In [ ]:
%%appyter code_eval

{% do DescriptionField(
    name='description',
    text='''The examples below were sourced from: 
<ul>
    <li><a href="http://tissues.jensenlab.org/Downloads" target="_blank">tissues.jensenlab.org</a> 
    <li><a href="http://diseases.jensenlab.org/Downloads" target="_blank">diseases.jensenlab.org</a> 
    <li><a href="http://compartments.jensenlab.org/Downloads" target="_blank">compartments.jensenlab.org</a>
</ul>
The compartment and tissue datasets both have one example, and the disease dataset has three examples. If clicking on the examples does not work, they should be downloaded directly from the source website. At least one channel must be provided.''',
    section='data'
) %}

{% set file_1 = FileField(constraint='(.*\.tsv$)?',
    name='file1', label='Channel 1 (tsv)', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    examples={
        'human_compartment_integrated_full.tsv (Compartments)': 'http://download.jensenlab.org/human_compartment_integrated_full.tsv',
        'human_disease_textmining_filtered.tsv (Diseases)': 'http://download.jensenlab.org/human_disease_textmining_filtered.tsv',
        'human_tissue_integrated_full.tsv (Tissues)': 'http://download.jensenlab.org/human_tissue_integrated_full.tsv'
    },
    section='data'
) %}

{% set file_2 = FileField(constraint='(.*\.tsv$)?', 
    name='file2', label='Channel 2 (tsv)', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    examples={
        'human_disease_knowledge_filtered.tsv (Diseases)': 'http://download.jensenlab.org/human_disease_knowledge_filtered.tsv'
    },
    section='data'
) %}

{% set file_3 = FileField(constraint='(.*\.tsv$)?', 
    name='file3', label='Channel 3 (tsv)', 
    default='Input/Jensen/human_disease_textmining_filtered.tsv',
    examples={
        'human_disease_experiments_filtered.tsv (Diseases)': 'http://download.jensenlab.org/human_disease_experiments_filtered.tsv'
    },
    section='data'
) %}


In [ ]:
%%appyter code_eval

{% set data = ChoiceField(
    name='group',
    label='Database',
    choices={
        'Compartments': 'Compartment',
        'Disease': 'Disease',
        'Tissue': 'Tissue'
    },
    default='Compartments',
    section='settings'
) %}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
%%appyter code_exec

output_name = 'jensen_{{data}}'.lower()

path = 'Output/Jensen-{{data}}'
if not os.path.exists(path):
    os.makedirs(path)

# Load Data

In [ ]:
%%appyter code_exec

file_list = [
    {{file_1}}, 
    {{file_2}}, 
    {{file_3}}
]

df_list = []
for f in file_list:
    if f:
        df_list.append(pd.read_csv(f, sep='\t', 
                        usecols=[1, 3], names=['Gene Symbol', '{{data}}'], 
                        index_col=0))
df = pd.concat(df_list)

In [ ]:
df.head()

In [ ]:
df.shape

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)